<a href="https://colab.research.google.com/github/psmouli14/final_project/blob/main/omniglot_prototype_combo_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import glob
from PIL import Image

import numpy as np
import tensorflow as tf

In [2]:
from google.colab import files
uploaded = files.upload()

Saving omniglot.zip to omniglot.zip


In [3]:
!unzip omniglot.zip

Streaming output truncated to the last 5000 lines.
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_05.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_11.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_10.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_04.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_12.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_06.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_07.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_13.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_17.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_03.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_02.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_16.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_14.png  


In [4]:
root_dir = 'omniglot/'
train_split_path = os.path.join(root_dir, 'splits', 'vinyals', 'train.txt')
print(train_split_path)

with open(train_split_path, 'r') as train_split:
    train_classes = [line.rstrip() for line in train_split.readlines()]
print(train_classes[2])

val_split_path = os.path.join(root_dir, 'splits', 'vinyals', 'val.txt')
print(val_split_path)

with open(val_split_path, 'r') as val_split:
    val_classes = [line.rstrip() for line in val_split.readlines()]
print(val_classes[2])

test_split_path = os.path.join(root_dir, 'splits', 'vinyals', 'test.txt')
print(test_split_path)

with open(test_split_path, 'r') as test_split:
    test_classes = [line.rstrip() for line in test_split.readlines()]
print(test_classes[2])

omniglot/splits/vinyals/train.txt
Angelic/character01/rot180
omniglot/splits/vinyals/val.txt
Hebrew/character01/rot180
omniglot/splits/vinyals/test.txt
Gurmukhi/character42/rot180


In [5]:
no_train_classes = len(train_classes)
print(no_train_classes)

no_val_classes = len(val_classes)
print(no_val_classes)

no_test_classes = len(test_classes)
print(no_test_classes)

4112
688
1692


In [6]:
#number of examples per class
num_examples = 20

#image width
img_width = 32

#image height
img_height = 32

#channels
channels = 1

In [7]:
train_dataset = np.zeros([no_train_classes, num_examples, img_height, img_width], dtype=np.float32)
print(train_dataset.shape)

val_dataset = np.zeros([no_val_classes, num_examples, img_height, img_width], dtype=np.float32)
print(val_dataset.shape)

test_dataset = np.zeros([no_test_classes, num_examples, img_height, img_width], dtype=np.float32)
print(test_dataset.shape)


(4112, 20, 32, 32)
(688, 20, 32, 32)
(1692, 20, 32, 32)


In [8]:
def populate_dataset(classes, dataset):
  for label, name in enumerate(classes):
    alphabet, character, rotation = name.split('/')
    rotation = float(rotation[3:])
    img_dir = os.path.join(root_dir, 'data', alphabet, character)
    img_files = sorted(glob.glob(os.path.join(img_dir, '*.png')))
    
    for index, img_file in enumerate(img_files):
      
      img = Image.open(img_file).resize((img_height, img_width)).rotate(rotation)
      img = np.asarray(img)
      img = 1 - img
      dataset[label, index] = img

  return dataset


train_dataset = populate_dataset(train_classes, train_dataset)
val_dataset = populate_dataset(val_classes, val_dataset)
test_dataset = populate_dataset(test_classes, test_dataset)


In [9]:
#Increase the axis by 1 to accomodate channel
train_dataset = np.expand_dims(train_dataset, axis=-1)
val_dataset = np.expand_dims(val_dataset, axis=-1)
test_dataset = np.expand_dims(test_dataset, axis=-1)

print(train_dataset.shape)
print(val_dataset.shape)
print(test_dataset.shape)

(4112, 20, 32, 32, 1)
(688, 20, 32, 32, 1)
(1692, 20, 32, 32, 1)


In [10]:
# This will be used to get the next set of support and query dataset for the episode
def get_next_episode(dataset, num_way, num_shot, num_query, no_of_classes):
  support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
  query = np.zeros([num_way, num_query, img_height, img_width, channels], dtype=np.float32)
  episodic_classes = np.random.permutation(no_of_classes)[:num_way]

  for index, class_ in enumerate(episodic_classes):
    selected = np.random.permutation(num_examples)[:num_shot + num_query]
    
    support[index] = dataset[class_][selected[:num_shot]]
    query[index] = dataset[class_][selected[num_shot:]]
    
  return support, query

def euclidean_distance(a, b):

    N, D = tf.shape(a)[0], tf.shape(a)[1]
    M = tf.shape(b)[0]
    a = tf.tile(tf.expand_dims(a, axis=1), (1, M, 1))
    b = tf.tile(tf.expand_dims(b, axis=0), (N, 1, 1))
    return tf.reduce_mean(tf.square(a - b), axis=2)


In [11]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, GlobalMaxPooling2D
from tensorflow.keras import Model

class Prototypical(Model):
   
    def __init__(self, n_support, n_query, w, h, c):
       
        super(Prototypical, self).__init__()
        self.w, self.h, self.c = w, h, c

        # Encoder of CNN with 4 blocks
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)), Flatten()]
        )

    def call(self, support, query):
        n_class = support.shape[0]
        n_support = support.shape[1]
        n_query = query.shape[1]
        y = np.tile(np.arange(n_class)[:, np.newaxis], (1, n_query))
        y_onehot = tf.cast(tf.one_hot(y, n_class), tf.float32)

        # correct indices of support samples (just natural order)
        target_inds = tf.reshape(tf.range(n_class), [n_class, 1])
        target_inds = tf.tile(target_inds, [1, n_query])

        # merge support and query to forward through encoder
        cat = tf.concat([
            tf.reshape(support, [n_class * n_support,
                                 self.w, self.h, self.c]),
            tf.reshape(query, [n_class * n_query,
                               self.w, self.h, self.c])], axis=0)
        z = self.encoder(cat)

        # Divide embedding into support and query
        z_prototypes = tf.reshape(z[:n_class * n_support],
                                  [n_class, n_support, z.shape[-1]])
        # Prototypes
        z_prototypes = tf.math.reduce_mean(z_prototypes, axis=1)
        z_query = z[n_class * n_support:]

        # Calculate distances between query and prototypes
        dists = euclidean_distance(z_query, z_prototypes)

        # log softmax 
        log_p_y = tf.nn.log_softmax(-dists, axis=-1)
        log_p_y = tf.reshape(log_p_y, [n_class, n_query, -1])
        
        loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_onehot, log_p_y), axis=-1), [-1]))
        eq = tf.cast(tf.equal(
            tf.cast(tf.argmax(log_p_y, axis=-1), tf.int32), 
            tf.cast(y, tf.int32)), tf.float32)
        acc = tf.reduce_mean(eq)
        return loss, acc

    def save(self, model_path):
        
        self.encoder.save(model_path)

    def load(self, model_path):
        
        self.encoder(tf.zeros([1, self.w, self.h, self.c]))
        self.encoder.load_weights(model_path)

In [12]:
# Lists to hold values for N-way k-shots experiments
train_num_ways = [60, 60, 40, 40]
test_num_ways = [5, 5, 20, 20]
num_shots = [5, 1, 5, 1]
learning_rate = 0.001

In [13]:
#Run prototypical model with only training set
# This will be using the first values from the lists for the experiment
num_epochs = 80
num_episodes = 100
save_path = "./results/models/omniglot_train0.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = train_num_ways[0] 

#number of examples per class for support set
num_shot = num_shots[0]  

#number of query points
num_query = num_shots[0] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  1.8223703
Epoch 1, Loss: 1.8223702907562256, Accuracy: 59.45664596557617


Saving new best model with loss:  0.6934691
Epoch 2, Loss: 0.6934691071510315, Accuracy: 80.90666198730469


Saving new best model with loss:  0.39470193
Epoch 3, Loss: 0.39470192790031433, Accuracy: 88.8933334350586


Saving new best model with loss:  0.26275072
Epoch 4, Loss: 0.2627507150173187, Accuracy: 92.49332427978516


Saving new best model with loss:  0.21018419
Epoch 5, Loss: 0.21018418669700623, Accuracy: 93.8800048828125


...

Saving new best model with loss:  0.043723036
Epoch 61, Loss: 0.04372303560376167, Accuracy: 98.51663208007812
Epoch 62, Loss: 0.046944886445999146, Accuracy: 98.50330352783203
Epoch 63, Loss: 0.04587273672223091, Accuracy: 98.4832992553711
Epoch 64, Loss: 0.049494363367557526, Accuracy: 98.31330871582031


Saving new best model with loss:  0.04034471
Epoch 65, Loss: 0.040344711393117905, Accuracy: 98.6032943725586
Epoch 66, Loss: 0.041141342371702194, Accuracy: 98.5533218383789
Epoch 67, Loss: 0.04525251314043999, Accuracy: 98.47332000732422
Epoch 68, Loss: 0.04823112487792969, Accuracy: 98.35997009277344


Saving new best model with loss:  0.038025767
Epoch 69, Loss: 0.03802576661109924, Accuracy: 98.75662994384766
Epoch 70, Loss: 0.04090440645813942, Accuracy: 98.61662292480469
Epoch 71, Loss: 0.04670336842536926, Accuracy: 98.43667602539062
Epoch 72, Loss: 0.0420367605984211, Accuracy: 98.60327911376953
Epoch 73, Loss: 0.04615261033177376, Accuracy: 98.45331573486328
Epoch 74, Loss: 0.04139649495482445, Accuracy: 98.61663818359375
Epoch 75, Loss: 0.04341845586895943, Accuracy: 98.6133041381836
Epoch 76, Loss: 0.0422544851899147, Accuracy: 98.52996063232422


Saving new best model with loss:  0.03801883
Epoch 77, Loss: 0.03801883012056351, Accuracy: 98.65997314453125


Saving new best model with loss:  0.036466975
Epoch 78, Loss: 0.03646697476506233, Accuracy: 98.76663208007812
Epoch 79, Loss: 0.03946603462100029, Accuracy: 98.6099624633789
Epoch 80, Loss: 0.04077218100428581, Accuracy: 98.5699691772461


In [14]:
#Run prototypical model with only training set
# This will be using the second values from the lists for the experiment
num_epochs = 80
num_episodes = 100
save_path = "./results/models/omniglot_train1.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = train_num_ways[1] 

#number of examples per class for support set
num_shot = num_shots[1]  

#number of query points
num_query = num_shots[1] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  2.725025
Epoch 1, Loss: 2.725024938583374, Accuracy: 38.58333206176758


Saving new best model with loss:  1.6814504
Epoch 2, Loss: 1.681450366973877, Accuracy: 56.53331756591797


Saving new best model with loss:  1.2530684
Epoch 3, Loss: 1.253068447113037, Accuracy: 67.20002746582031


Saving new best model with loss:  0.897805
Epoch 4, Loss: 0.8978049755096436, Accuracy: 74.76666259765625


Saving new best model with loss:  0.75266457
Epoch 5, Loss: 0.7526645660400391, Accuracy: 79.08333587646484


...

Saving new best model with loss:  0.20709206
Epoch 52, Loss: 0.2070920616388321, Accuracy: 94.01666259765625
Epoch 53, Loss: 0.22769029438495636, Accuracy: 93.38333129882812
Epoch 54, Loss: 0.21028442680835724, Accuracy: 94.04998016357422
Epoch 55, Loss: 0.2184450775384903, Accuracy: 93.55001068115234


Saving new best model with loss:  0.19155377
Epoch 56, Loss: 0.1915537714958191, Accuracy: 94.11666107177734
Epoch 57, Loss: 0.21993903815746307, Accuracy: 93.16667938232422
Epoch 58, Loss: 0.1945711374282837, Accuracy: 94.3833236694336
Epoch 59, Loss: 0.19814616441726685, Accuracy: 93.93331146240234
Epoch 60, Loss: 0.19905954599380493, Accuracy: 93.94999694824219
Epoch 61, Loss: 0.20597705245018005, Accuracy: 94.13331604003906
Epoch 62, Loss: 0.21292129158973694, Accuracy: 93.96666717529297
Epoch 63, Loss: 0.1921580731868744, Accuracy: 94.24998474121094


Saving new best model with loss:  0.18370558
Epoch 64, Loss: 0.18370558321475983, Accuracy: 94.36666870117188
Epoch 65, Loss: 0.2319098263978958, Accuracy: 93.04999542236328
Epoch 66, Loss: 0.1956406980752945, Accuracy: 93.95001220703125
Epoch 67, Loss: 0.19223757088184357, Accuracy: 93.94998168945312
Epoch 68, Loss: 0.18465524911880493, Accuracy: 94.24998474121094
Epoch 69, Loss: 0.18447345495224, Accuracy: 94.06665802001953


Saving new best model with loss:  0.1738064
Epoch 70, Loss: 0.17380639910697937, Accuracy: 94.6333236694336
Epoch 71, Loss: 0.19733457267284393, Accuracy: 94.23332977294922
Epoch 72, Loss: 0.1998845338821411, Accuracy: 94.13331604003906
Epoch 73, Loss: 0.180770143866539, Accuracy: 94.68331909179688
Epoch 74, Loss: 0.1760677546262741, Accuracy: 94.43331146240234
Epoch 75, Loss: 0.18751540780067444, Accuracy: 94.19998931884766
Epoch 76, Loss: 0.1813250184059143, Accuracy: 94.46664428710938
Epoch 77, Loss: 0.1927918940782547, Accuracy: 94.04999542236328
Epoch 78, Loss: 0.18336202204227448, Accuracy: 94.38330841064453
Epoch 79, Loss: 0.19591639935970306, Accuracy: 93.93334197998047


Saving new best model with loss:  0.16431047
Epoch 80, Loss: 0.16431047022342682, Accuracy: 95.36666107177734


In [15]:
#Run prototypical model with only training set
# This will be using the third values from the lists for the experiment
num_epochs = 80
num_episodes = 100
save_path = "./results/models/omniglot_train2.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = train_num_ways[2] 

#number of examples per class for support set
num_shot = num_shots[2]  

#number of query points
num_query = num_shots[2] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  1.6227574
Epoch 1, Loss: 1.6227574348449707, Accuracy: 62.99502182006836


Saving new best model with loss:  0.61009717
Epoch 2, Loss: 0.6100971698760986, Accuracy: 83.37998962402344


Saving new best model with loss:  0.40875885
Epoch 3, Loss: 0.40875884890556335, Accuracy: 88.61997985839844


Saving new best model with loss:  0.2541982
Epoch 4, Loss: 0.25419819355010986, Accuracy: 92.61499786376953


Saving new best model with loss:  0.2009862
Epoch 5, Loss: 0.20098620653152466, Accuracy: 94.19498443603516


...

Saving new best model with loss:  0.03624097
Epoch 66, Loss: 0.036240968853235245, Accuracy: 98.7650146484375
Epoch 67, Loss: 0.03726862370967865, Accuracy: 98.78002166748047
Epoch 68, Loss: 0.03950682654976845, Accuracy: 98.66002655029297


Saving new best model with loss:  0.036107734
Epoch 69, Loss: 0.036107733845710754, Accuracy: 98.85003662109375
Epoch 70, Loss: 0.0373772457242012, Accuracy: 98.635009765625
Epoch 71, Loss: 0.037399400025606155, Accuracy: 98.81500244140625


Saving new best model with loss:  0.034875084
Epoch 72, Loss: 0.03487508371472359, Accuracy: 98.88504028320312
Epoch 73, Loss: 0.04054630175232887, Accuracy: 98.74002838134766


Saving new best model with loss:  0.032696187
Epoch 74, Loss: 0.0326961874961853, Accuracy: 98.95502471923828
Epoch 75, Loss: 0.035186897963285446, Accuracy: 98.81002807617188
Epoch 76, Loss: 0.034691277891397476, Accuracy: 98.7550277709961
Epoch 77, Loss: 0.034321557730436325, Accuracy: 98.96003723144531


Saving new best model with loss:  0.031905357
Epoch 78, Loss: 0.03190535679459572, Accuracy: 98.91503143310547
Epoch 79, Loss: 0.03580688312649727, Accuracy: 98.86001586914062
Epoch 80, Loss: 0.03340233862400055, Accuracy: 98.90503692626953


In [16]:
#Run prototypical model with only training set
# This will be using the fourth values from the lists for the experiment
num_epochs = 80
num_episodes = 100
save_path = "./results/models/omniglot_train3.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = train_num_ways[3] 

#number of examples per class for support set
num_shot = num_shots[3]  

#number of query points
num_query = num_shots[3] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  2.4195518
Epoch 1, Loss: 2.4195518493652344, Accuracy: 41.549991607666016


Saving new best model with loss:  1.5646489
Epoch 2, Loss: 1.5646488666534424, Accuracy: 58.124996185302734


Saving new best model with loss:  1.197028
Epoch 3, Loss: 1.1970280408859253, Accuracy: 68.25001525878906


Saving new best model with loss:  0.93336415
Epoch 4, Loss: 0.9333641529083252, Accuracy: 74.8499984741211


Saving new best model with loss:  0.7554164
Epoch 5, Loss: 0.7554163932800293, Accuracy: 78.92499542236328


...

Saving new best model with loss:  0.19183627
Epoch 51, Loss: 0.19183626770973206, Accuracy: 94.32498931884766
Epoch 52, Loss: 0.21516595780849457, Accuracy: 93.57498168945312
Epoch 53, Loss: 0.22133274376392365, Accuracy: 92.94996643066406


Saving new best model with loss:  0.18847893
Epoch 54, Loss: 0.18847893178462982, Accuracy: 94.22498321533203
Epoch 55, Loss: 0.21808242797851562, Accuracy: 93.49995422363281
Epoch 56, Loss: 0.20452998578548431, Accuracy: 93.72500610351562
Epoch 57, Loss: 0.20025140047073364, Accuracy: 94.07499694824219
Epoch 58, Loss: 0.20591992139816284, Accuracy: 93.64998626708984
Epoch 59, Loss: 0.20623888075351715, Accuracy: 93.9749984741211
Epoch 60, Loss: 0.19467797875404358, Accuracy: 94.39998626708984


Saving new best model with loss:  0.18176337
Epoch 61, Loss: 0.18176336586475372, Accuracy: 94.02497100830078
Epoch 62, Loss: 0.18789023160934448, Accuracy: 94.60000610351562
Epoch 63, Loss: 0.1919507533311844, Accuracy: 94.59999084472656
Epoch 64, Loss: 0.18456722795963287, Accuracy: 94.67495727539062


Saving new best model with loss:  0.1750999
Epoch 65, Loss: 0.17509989440441132, Accuracy: 94.5999755859375


Saving new best model with loss:  0.15712206
Epoch 66, Loss: 0.15712206065654755, Accuracy: 95.57496643066406
Epoch 67, Loss: 0.17420987784862518, Accuracy: 94.52496337890625
Epoch 68, Loss: 0.18387925624847412, Accuracy: 94.87496948242188
Epoch 69, Loss: 0.17550747096538544, Accuracy: 94.5999755859375
Epoch 70, Loss: 0.17219987511634827, Accuracy: 94.47496795654297
Epoch 71, Loss: 0.18452712893486023, Accuracy: 94.57498168945312
Epoch 72, Loss: 0.19801875948905945, Accuracy: 94.59998321533203
Epoch 73, Loss: 0.17818252742290497, Accuracy: 94.50000762939453
Epoch 74, Loss: 0.19622284173965454, Accuracy: 93.90000915527344
Epoch 75, Loss: 0.17716480791568756, Accuracy: 94.5999755859375
Epoch 76, Loss: 0.1686609983444214, Accuracy: 94.69996643066406
Epoch 77, Loss: 0.19023028016090393, Accuracy: 94.74999237060547
Epoch 78, Loss: 0.16630807518959045, Accuracy: 95.19995880126953
Epoch 79, Loss: 0.1677059680223465, Accuracy: 94.94998168945312
Epoch 80, Loss: 0.16286741197109222, Accuracy: 95

In [17]:
save_paths = ["./results/models/omniglot_train0.h5", "./results/models/omniglot_train1.h5", "./results/models/omniglot_train2.h5", "./results/models/omniglot_train3.h5"]

In [18]:
def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

In [19]:
accuracies_proto_0 = {}
for save_path in save_paths:
  model_path = save_path
  model.load(model_path)
  print("Model loaded.")
  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[0]

  #number of examples per class for support set
  num_shot = num_shots[0]  

  #number of query points
  num_query = num_shots[0] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')


  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_proto_0[model_path] = accuracy


Model loaded.
Loss:  0.01443269
Accuracy:  99.65207576751709
Model loaded.
Loss:  0.017238224
Accuracy:  99.6001124382019
Model loaded.
Loss:  0.022710524
Accuracy:  99.52009320259094
Model loaded.
Loss:  0.017339597
Accuracy:  99.56807494163513


In [20]:
accuracies_proto_1 = {}
for save_path in save_paths:
  model_path = save_path
  model.load(model_path)
  print("Model with path {} loaded.".format(save_path))
  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[1]

  #number of examples per class for support set
  num_shot = num_shots[1]  

  #number of query points
  num_query = num_shots[1] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')


  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_proto_1[model_path] = accuracy


Model with path ./results/models/omniglot_train0.h5 loaded.
Loss:  0.10468974
Accuracy:  97.63989448547363
Model with path ./results/models/omniglot_train1.h5 loaded.
Loss:  0.064977564
Accuracy:  98.13992977142334
Model with path ./results/models/omniglot_train2.h5 loaded.
Loss:  0.122516006
Accuracy:  97.03988432884216
Model with path ./results/models/omniglot_train3.h5 loaded.
Loss:  0.06688529
Accuracy:  98.11992049217224


In [21]:
accuracies_proto_2 = {}
for save_path in save_paths:
  model_path = save_path
  model.load(model_path)

  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[2]

  #number of examples per class for support set
  num_shot = num_shots[2]  

  #number of query points
  num_query = num_shots[2] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')

  print("Testing {} way {} shot".format(num_way, num_shot))
  print("Model with path {} loaded.".format(save_path))

  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_proto_2[model_path] = accuracy

Model with path ./results/models/omniglot_train0.h5 loaded.
Loss:  0.049333546
Accuracy:  98.63952994346619
Model with path ./results/models/omniglot_train1.h5 loaded.
Loss:  0.050810993
Accuracy:  98.72457981109619
Model with path ./results/models/omniglot_train2.h5 loaded.
Loss:  0.060099393
Accuracy:  98.33952188491821
Model with path ./results/models/omniglot_train3.h5 loaded.
Loss:  0.059270512
Accuracy:  98.42256903648376


In [22]:
accuracies_proto_3 = {}

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for save_path in save_paths:
  model_path = save_path
  model.load(model_path)
  print("Model with path {} loaded.".format(save_path))
  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[3]

  #number of examples per class for support set
  num_shot = num_shots[3]  

  #number of query points
  num_query = num_shots[3] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')


  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_proto_3[model_path] = accuracy

Model with path ./results/models/omniglot_train0.h5 loaded.
Loss:  0.3221269
Accuracy:  92.59039759635925
Model with path ./results/models/omniglot_train1.h5 loaded.
Loss:  0.16637978
Accuracy:  95.2004075050354
Model with path ./results/models/omniglot_train2.h5 loaded.
Loss:  0.3385203
Accuracy:  91.7953372001648
Model with path ./results/models/omniglot_train3.h5 loaded.
Loss:  0.23058069
Accuracy:  93.64041090011597


In [23]:
#Combi Proto + Reptile for first values from the list ie 60-way 5-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = train_num_ways[0]

#number of examples per class for support set
num_shot = num_shots[0]  

#number of query points
num_query = num_shots[0] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate, beta_1=0)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 241
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile0.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  val_loss(loss)
  val_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  val_loss.reset_states()
  train_acc.reset_states()
  val_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    
    model = Prototypical(train_support, train_query, img_width, img_height, channels)
    model.call(train_support, train_query)
    old_weights = model.get_weights()
    train_step(train_support, train_query, optimizer_sgd)
    new_weights = model.get_weights()

    for part_weight in range(len(new_weights)):
        new_weights[part_weight] = old_weights[part_weight] + (
            (new_weights[part_weight] - old_weights[part_weight]) * cur_meta_step_size
        )

    model.set_weights(new_weights)
    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_adam)
      model.set_weights(old_vars)
      
  cur_loss = val_loss.result().numpy()

  if (epoch % eval_interval == 0):
    template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, val_loss.result(),
                            val_acc.result() * 100))
    if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)

Epoch 1, Loss: 1.5062423944473267, Accuracy: 65.39667510986328, Val Loss: 1.534399390220642, Val Accuracy: 65.55000305175781
Saving new best model with loss:  1.5343994


Epoch 5, Loss: 0.5645511746406555, Accuracy: 84.56001281738281, Val Loss: 0.5675460696220398, Val Accuracy: 84.68998718261719
Saving new best model with loss:  0.56754607


Epoch 9, Loss: 0.3253828287124634, Accuracy: 90.81998443603516, Val Loss: 0.33602410554885864, Val Accuracy: 90.38001251220703
Saving new best model with loss:  0.3360241


Epoch 13, Loss: 0.25109007954597473, Accuracy: 92.83332061767578, Val Loss: 0.2403283715248108, Val Accuracy: 93.19664764404297
Saving new best model with loss:  0.24032837


Epoch 17, Loss: 0.21814675629138947, Accuracy: 93.75997924804688, Val Loss: 0.2108759880065918, Val Accuracy: 93.95664978027344
Saving new best model with loss:  0.21087599


...

Epoch 173, Loss: 0.06476714462041855, Accuracy: 98.04664611816406, Val Loss: 0.05787741765379906, Val Accuracy: 98.11000061035156
Saving new best model with loss:  0.057877418


Epoch 177, Loss: 0.05526799336075783, Accuracy: 98.25332641601562, Val Loss: 0.0575527586042881, Val Accuracy: 98.23998260498047
Saving new best model with loss:  0.05755276
Epoch 181, Loss: 0.05736425891518593, Accuracy: 98.15664672851562, Val Loss: 0.06108621135354042, Val Accuracy: 98.10002136230469
Epoch 185, Loss: 0.0550750195980072, Accuracy: 98.20999145507812, Val Loss: 0.06354694813489914, Val Accuracy: 97.97332000732422


Epoch 189, Loss: 0.054399359971284866, Accuracy: 98.22333526611328, Val Loss: 0.05400128662586212, Val Accuracy: 98.35999298095703
Saving new best model with loss:  0.054001287


Epoch 193, Loss: 0.054537512362003326, Accuracy: 98.37665557861328, Val Loss: 0.0524386502802372, Val Accuracy: 98.36331176757812
Saving new best model with loss:  0.05243865
Epoch 197, Loss: 0.053780246526002884, Accuracy: 98.28997802734375, Val Loss: 0.05883496254682541, Val Accuracy: 98.07667541503906
Epoch 201, Loss: 0.05303064361214638, Accuracy: 98.3299560546875, Val Loss: 0.055257946252822876, Val Accuracy: 98.29666137695312


Epoch 205, Loss: 0.052976205945014954, Accuracy: 98.39665985107422, Val Loss: 0.04552629962563515, Val Accuracy: 98.51995849609375
Saving new best model with loss:  0.0455263
Epoch 209, Loss: 0.051373306661844254, Accuracy: 98.35332489013672, Val Loss: 0.05278395116329193, Val Accuracy: 98.2933349609375
Epoch 213, Loss: 0.05315212160348892, Accuracy: 98.23663330078125, Val Loss: 0.05116524547338486, Val Accuracy: 98.29663848876953
Epoch 217, Loss: 0.04985051229596138, Accuracy: 98.48330688476562, Val Loss: 0.04787299782037735, Val Accuracy: 98.36997985839844
Epoch 221, Loss: 0.048713792115449905, Accuracy: 98.30663299560547, Val Loss: 0.0480974055826664, Val Accuracy: 98.42664337158203
Epoch 225, Loss: 0.049977127462625504, Accuracy: 98.37332153320312, Val Loss: 0.05241459235548973, Val Accuracy: 98.35330200195312
Epoch 229, Loss: 0.050369992852211, Accuracy: 98.35665893554688, Val Loss: 0.046472545713186264, Val Accuracy: 98.47332763671875
Epoch 233, Loss: 0.04515768587589264, Accurac

In [24]:
#Combi Proto + Reptile for second values from the list ie 60-way 1-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = train_num_ways[1]

#number of examples per class for support set
num_shot = num_shots[1]  

#number of query points
num_query = num_shots[1] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate, beta_1=0)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 241
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile1.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  val_loss(loss)
  val_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  val_loss.reset_states()
  train_acc.reset_states()
  val_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    
    model = Prototypical(train_support, train_query, img_width, img_height, channels)
    model.call(train_support, train_query)
    old_weights = model.get_weights()
    train_step(train_support, train_query, optimizer_sgd)
    new_weights = model.get_weights()

    for part_weight in range(len(new_weights)):
        new_weights[part_weight] = old_weights[part_weight] + (
            (new_weights[part_weight] - old_weights[part_weight]) * cur_meta_step_size
        )

    model.set_weights(new_weights)
    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_adam)
      model.set_weights(old_vars)
      
  cur_loss = val_loss.result().numpy()

  if (epoch % eval_interval == 0):
    template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, val_loss.result(),
                            val_acc.result() * 100))
    if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)

Epoch 1, Loss: 2.6033387184143066, Accuracy: 40.98332977294922, Val Loss: 2.576746940612793, Val Accuracy: 41.04999923706055
Saving new best model with loss:  2.576747


Epoch 5, Loss: 1.7560092210769653, Accuracy: 56.23332214355469, Val Loss: 1.7480398416519165, Val Accuracy: 55.94999313354492
Saving new best model with loss:  1.7480398


Epoch 9, Loss: 1.218807339668274, Accuracy: 68.01668548583984, Val Loss: 1.1998348236083984, Val Accuracy: 68.35002136230469
Saving new best model with loss:  1.1998348


Epoch 13, Loss: 0.8729893565177917, Accuracy: 75.8833236694336, Val Loss: 0.9005581140518188, Val Accuracy: 75.55000305175781
Saving new best model with loss:  0.9005581


Epoch 17, Loss: 0.7501782774925232, Accuracy: 79.44998931884766, Val Loss: 0.7413337826728821, Val Accuracy: 79.44998931884766
Saving new best model with loss:  0.7413338


...

Epoch 209, Loss: 0.22516338527202606, Accuracy: 92.98333740234375, Val Loss: 0.2149752825498581, Val Accuracy: 93.36665344238281
Saving new best model with loss:  0.21497528


Epoch 213, Loss: 0.20225901901721954, Accuracy: 93.86664581298828, Val Loss: 0.21099480986595154, Val Accuracy: 93.80001068115234
Saving new best model with loss:  0.21099481
Epoch 217, Loss: 0.24796682596206665, Accuracy: 92.93333435058594, Val Loss: 0.23946760594844818, Val Accuracy: 92.98332214355469


Epoch 221, Loss: 0.22618892788887024, Accuracy: 93.33332061767578, Val Loss: 0.20968016982078552, Val Accuracy: 93.78334045410156
Saving new best model with loss:  0.20968017
Epoch 225, Loss: 0.20451462268829346, Accuracy: 93.4666519165039, Val Loss: 0.2385098785161972, Val Accuracy: 93.05000305175781


Epoch 229, Loss: 0.1802801936864853, Accuracy: 94.3499984741211, Val Loss: 0.2032293826341629, Val Accuracy: 94.04998016357422
Saving new best model with loss:  0.20322938
Epoch 233, Loss: 0.21104231476783752, Accuracy: 93.81666564941406, Val Loss: 0.2121756225824356, Val Accuracy: 93.78331756591797
Epoch 237, Loss: 0.22431673109531403, Accuracy: 93.73332977294922, Val Loss: 0.21172069013118744, Val Accuracy: 93.55001068115234


Epoch 241, Loss: 0.2029552310705185, Accuracy: 94.0333251953125, Val Loss: 0.20284609496593475, Val Accuracy: 93.88333129882812
Saving new best model with loss:  0.2028461


In [25]:
#Combi Proto + Reptile for first values from the list ie 40-way 5-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = train_num_ways[2]

#number of examples per class for support set
num_shot = num_shots[2]  

#number of query points
num_query = num_shots[2] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate, beta_1=0)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 241
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile2.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  val_loss(loss)
  val_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  val_loss.reset_states()
  train_acc.reset_states()
  val_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    
    model = Prototypical(train_support, train_query, img_width, img_height, channels)
    model.call(train_support, train_query)
    old_weights = model.get_weights()
    train_step(train_support, train_query, optimizer_sgd)
    new_weights = model.get_weights()

    for part_weight in range(len(new_weights)):
        new_weights[part_weight] = old_weights[part_weight] + (
            (new_weights[part_weight] - old_weights[part_weight]) * cur_meta_step_size
        )

    model.set_weights(new_weights)
    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_adam)
      model.set_weights(old_vars)
      
  cur_loss = val_loss.result().numpy()

  if (epoch % eval_interval == 0):
    template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, val_loss.result(),
                            val_acc.result() * 100))
    if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)

Epoch 1, Loss: 1.2778325080871582, Accuracy: 70.7650146484375, Val Loss: 1.2833958864212036, Val Accuracy: 70.57499694824219
Saving new best model with loss:  1.2833959


Epoch 5, Loss: 0.531635046005249, Accuracy: 85.2649917602539, Val Loss: 0.5366384387016296, Val Accuracy: 85.49000549316406
Saving new best model with loss:  0.53663844


Epoch 9, Loss: 0.3411201536655426, Accuracy: 90.114990234375, Val Loss: 0.3406173288822174, Val Accuracy: 90.59001922607422
Saving new best model with loss:  0.34061733


Epoch 13, Loss: 0.24660103023052216, Accuracy: 92.8499755859375, Val Loss: 0.2399563193321228, Val Accuracy: 93.18499755859375
Saving new best model with loss:  0.23995632


Epoch 17, Loss: 0.19253690540790558, Accuracy: 94.36499786376953, Val Loss: 0.18862922489643097, Val Accuracy: 94.62498474121094
Saving new best model with loss:  0.18862922


...

Epoch 173, Loss: 0.05552484840154648, Accuracy: 98.34504699707031, Val Loss: 0.046321965754032135, Val Accuracy: 98.59003448486328
Saving new best model with loss:  0.046321966
Epoch 177, Loss: 0.05484458804130554, Accuracy: 98.32502746582031, Val Loss: 0.05130770802497864, Val Accuracy: 98.33003997802734
Epoch 181, Loss: 0.05601535737514496, Accuracy: 98.27001953125, Val Loss: 0.05257531255483627, Val Accuracy: 98.25501251220703
Epoch 185, Loss: 0.05296824499964714, Accuracy: 98.3900375366211, Val Loss: 0.05188318341970444, Val Accuracy: 98.42002868652344
Epoch 189, Loss: 0.048764124512672424, Accuracy: 98.43000793457031, Val Loss: 0.04891091212630272, Val Accuracy: 98.54502868652344
Epoch 193, Loss: 0.05324413254857063, Accuracy: 98.46501159667969, Val Loss: 0.0491485558450222, Val Accuracy: 98.5250015258789


Epoch 197, Loss: 0.05165988579392433, Accuracy: 98.42501068115234, Val Loss: 0.04530874267220497, Val Accuracy: 98.48002624511719
Saving new best model with loss:  0.045308743
Epoch 201, Loss: 0.04475121200084686, Accuracy: 98.55001831054688, Val Loss: 0.04991777241230011, Val Accuracy: 98.3900146484375


Epoch 205, Loss: 0.05244136229157448, Accuracy: 98.34001922607422, Val Loss: 0.04506022483110428, Val Accuracy: 98.54503631591797
Saving new best model with loss:  0.045060225
Epoch 209, Loss: 0.053685903549194336, Accuracy: 98.44002532958984, Val Loss: 0.04774003475904465, Val Accuracy: 98.53001403808594
Epoch 213, Loss: 0.04466458410024643, Accuracy: 98.59001922607422, Val Loss: 0.051700323820114136, Val Accuracy: 98.6250228881836
Epoch 217, Loss: 0.04466061666607857, Accuracy: 98.56502532958984, Val Loss: 0.046057119965553284, Val Accuracy: 98.54002380371094
Epoch 221, Loss: 0.04654303193092346, Accuracy: 98.51502227783203, Val Loss: 0.04824216291308403, Val Accuracy: 98.49504089355469


Epoch 225, Loss: 0.04849007725715637, Accuracy: 98.60501861572266, Val Loss: 0.04121449589729309, Val Accuracy: 98.7650375366211
Saving new best model with loss:  0.041214496


Epoch 229, Loss: 0.045249711722135544, Accuracy: 98.55001831054688, Val Loss: 0.039442431181669235, Val Accuracy: 98.68502044677734
Saving new best model with loss:  0.03944243
Epoch 233, Loss: 0.040863752365112305, Accuracy: 98.63504028320312, Val Loss: 0.04529554396867752, Val Accuracy: 98.59504699707031
Epoch 237, Loss: 0.047326575964689255, Accuracy: 98.42503356933594, Val Loss: 0.04339262843132019, Val Accuracy: 98.6600341796875
Epoch 241, Loss: 0.04493933171033859, Accuracy: 98.44500732421875, Val Loss: 0.04194182530045509, Val Accuracy: 98.68501281738281


In [26]:
#Combi Proto + Reptile for first values from the list ie 40-way 1-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = train_num_ways[3]

#number of examples per class for support set
num_shot = num_shots[3]  

#number of query points
num_query = num_shots[3] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate, beta_1=0)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 241
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile3.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  val_loss(loss)
  val_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  val_loss.reset_states()
  train_acc.reset_states()
  val_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    
    model = Prototypical(train_support, train_query, img_width, img_height, channels)
    model.call(train_support, train_query)
    old_weights = model.get_weights()
    train_step(train_support, train_query, optimizer_sgd)
    new_weights = model.get_weights()

    for part_weight in range(len(new_weights)):
        new_weights[part_weight] = old_weights[part_weight] + (
            (new_weights[part_weight] - old_weights[part_weight]) * cur_meta_step_size
        )

    model.set_weights(new_weights)
    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_adam)
      model.set_weights(old_vars)
      
  cur_loss = val_loss.result().numpy()

  if (epoch % eval_interval == 0):
    template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, val_loss.result(),
                            val_acc.result() * 100))
    if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)

Epoch 1, Loss: 2.2832324504852295, Accuracy: 45.125, Val Loss: 2.2794322967529297, Val Accuracy: 46.64999771118164
Saving new best model with loss:  2.2794323


Epoch 5, Loss: 1.5152398347854614, Accuracy: 61.07499694824219, Val Loss: 1.4932222366333008, Val Accuracy: 62.350006103515625
Saving new best model with loss:  1.4932222


Epoch 9, Loss: 1.1236075162887573, Accuracy: 70.52499389648438, Val Loss: 1.1055552959442139, Val Accuracy: 69.07499694824219
Saving new best model with loss:  1.1055553


Epoch 13, Loss: 0.7904539704322815, Accuracy: 78.07499694824219, Val Loss: 0.7906769514083862, Val Accuracy: 77.92498779296875
Saving new best model with loss:  0.79067695


Epoch 17, Loss: 0.7175846695899963, Accuracy: 79.79999542236328, Val Loss: 0.6741983294487, Val Accuracy: 80.9000015258789
Saving new best model with loss:  0.6741983


...

Epoch 161, Loss: 0.25193464756011963, Accuracy: 92.79998779296875, Val Loss: 0.2179429680109024, Val Accuracy: 93.27497863769531
Saving new best model with loss:  0.21794297


Epoch 165, Loss: 0.22168253362178802, Accuracy: 93.45000457763672, Val Loss: 0.20432928204536438, Val Accuracy: 93.60000610351562
Saving new best model with loss:  0.20432928
Epoch 169, Loss: 0.23721417784690857, Accuracy: 92.84998321533203, Val Loss: 0.20978718996047974, Val Accuracy: 93.67501068115234
Epoch 173, Loss: 0.2343026101589203, Accuracy: 93.32502746582031, Val Loss: 0.23101498186588287, Val Accuracy: 93.29999542236328
Epoch 177, Loss: 0.23195543885231018, Accuracy: 93.09999084472656, Val Loss: 0.22966672480106354, Val Accuracy: 93.80001068115234
Epoch 181, Loss: 0.21852423250675201, Accuracy: 93.84996795654297, Val Loss: 0.2323383390903473, Val Accuracy: 92.87500762939453
Epoch 185, Loss: 0.22868211567401886, Accuracy: 93.57498168945312, Val Loss: 0.24987754225730896, Val Accuracy: 92.89997863769531
Epoch 189, Loss: 0.21965712308883667, Accuracy: 93.04997253417969, Val Loss: 0.2370077520608902, Val Accuracy: 92.5999984741211
Epoch 193, Loss: 0.2252950221300125, Accuracy: 93

Epoch 197, Loss: 0.23177404701709747, Accuracy: 93.77497863769531, Val Loss: 0.19102539122104645, Val Accuracy: 94.3249740600586
Saving new best model with loss:  0.19102539


Epoch 201, Loss: 0.22074775397777557, Accuracy: 93.6500015258789, Val Loss: 0.18636654317378998, Val Accuracy: 94.52497100830078
Saving new best model with loss:  0.18636654
Epoch 205, Loss: 0.2062712460756302, Accuracy: 93.77498626708984, Val Loss: 0.2193373143672943, Val Accuracy: 93.49998474121094
Epoch 209, Loss: 0.19275163114070892, Accuracy: 94.29998779296875, Val Loss: 0.1966104954481125, Val Accuracy: 94.27497863769531
Epoch 213, Loss: 0.22827354073524475, Accuracy: 93.57501220703125, Val Loss: 0.2046828418970108, Val Accuracy: 93.97500610351562
Epoch 217, Loss: 0.22473737597465515, Accuracy: 93.79998779296875, Val Loss: 0.2085060477256775, Val Accuracy: 93.82498168945312
Epoch 221, Loss: 0.18876248598098755, Accuracy: 94.44995880126953, Val Loss: 0.19851471483707428, Val Accuracy: 93.90001678466797
Epoch 225, Loss: 0.20458747446537018, Accuracy: 94.22498321533203, Val Loss: 0.19220851361751556, Val Accuracy: 94.22494506835938
Epoch 229, Loss: 0.16747373342514038, Accuracy: 95.

Epoch 241, Loss: 0.18552295863628387, Accuracy: 94.40000915527344, Val Loss: 0.18530941009521484, Val Accuracy: 94.57499694824219
Saving new best model with loss:  0.18530941


In [27]:
reptile_save_paths = ["./results/models/omniglot_train_reptile0.h5", "./results/models/omniglot_train_reptile1.h5", \
                      "./results/models/omniglot_train_reptile2.h5",\
                      "./results/models/omniglot_train_reptile3.h5"]

In [34]:
accuracies_reptile_0 = {}
support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)

def calc_loss(model, support, query):
  loss, acc = model.call(support, query)
  return loss, acc

for save_path in reptile_save_paths:
  model_path = save_path
  model.load(model_path)
  print("Model with path {} loaded.".format(save_path))
  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[0]

  #number of examples per class for support set
  num_shot = num_shots[0]  

  #number of query points
  num_query = num_shots[0] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')


  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(model, test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_reptile_0[model_path] = accuracy

Model with path ./results/models/omniglot_train_reptile0.h5 loaded.
Loss:  1.6092929
Accuracy:  82.28806853294373
Model with path ./results/models/omniglot_train_reptile1.h5 loaded.
Loss:  1.6091865
Accuracy:  81.18806481361389
Model with path ./results/models/omniglot_train_reptile2.h5 loaded.
Loss:  1.6092588
Accuracy:  82.27205872535706
Model with path ./results/models/omniglot_train_reptile3.h5 loaded.
Loss:  1.6092482
Accuracy:  81.26807808876038


In [35]:
accuracies_reptile_1 = {}

def calc_loss(model, support, query):
  loss, acc = model.call(support, query)
  return loss, acc

for save_path in reptile_save_paths:
  model_path = save_path
  model.load(model_path)
  print("Model with path {} loaded.".format(save_path))
  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[1]

  #number of examples per class for support set
  num_shot = num_shots[1]  

  #number of query points
  num_query = num_shots[1] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')

  print("Testing {} way {} shot".format(num_way, num_shot))

  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(model, test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_reptile_1[model_path] = accuracy

Model with path ./results/models/omniglot_train_reptile0.h5 loaded.
Testing 5 way 1 shot
Loss:  1.6092882
Accuracy:  63.75981569290161
Model with path ./results/models/omniglot_train_reptile1.h5 loaded.
Testing 5 way 1 shot
Loss:  1.6091793
Accuracy:  64.97978568077087
Model with path ./results/models/omniglot_train_reptile2.h5 loaded.
Testing 5 way 1 shot
Loss:  1.6092618
Accuracy:  65.17989039421082
Model with path ./results/models/omniglot_train_reptile3.h5 loaded.
Testing 5 way 1 shot
Loss:  1.6092479
Accuracy:  63.45980763435364


In [36]:
accuracies_reptile_2 = {}

def calc_loss(model, support, query):
  loss, acc = model.call(support, query)
  return loss, acc

for save_path in reptile_save_paths:
  model_path = save_path
  model.load(model_path)
  print("Model with path {} loaded.".format(save_path))
  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[2]

  #number of examples per class for support set
  num_shot = num_shots[2]  

  #number of query points
  num_query = num_shots[2] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')

  print("Testing {} way {} shot".format(num_way, num_shot))

  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(model, test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_reptile_2[model_path] = accuracy

Model with path ./results/models/omniglot_train_reptile0.h5 loaded.
Testing 20 way 5 shot
Loss:  2.9955788
Accuracy:  64.20199871063232
Model with path ./results/models/omniglot_train_reptile1.h5 loaded.
Testing 20 way 5 shot
Loss:  2.9954238
Accuracy:  61.94004416465759
Model with path ./results/models/omniglot_train_reptile2.h5 loaded.
Testing 20 way 5 shot
Loss:  2.995539
Accuracy:  63.39397430419922
Model with path ./results/models/omniglot_train_reptile3.h5 loaded.
Testing 20 way 5 shot
Loss:  2.9955156
Accuracy:  61.36692762374878


In [37]:
accuracies_reptile_3 = {}

def calc_loss(model, support, query):
  loss, acc = model.call(support, query)
  return loss, acc

for save_path in reptile_save_paths:
  model_path = save_path
  model.load(model_path)
  print("Model with path {} loaded.".format(save_path))
  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[3]

  #number of examples per class for support set
  num_shot = num_shots[3]  

  #number of query points
  num_query = num_shots[3] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')

  print("Testing {} way {} shot".format(num_way, num_shot))

  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(model, test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_reptile_3[model_path] = accuracy

Model with path ./results/models/omniglot_train_reptile0.h5 loaded.
Testing 20 way 1 shot
Loss:  2.9955764
Accuracy:  41.069960594177246
Model with path ./results/models/omniglot_train_reptile1.h5 loaded.
Testing 20 way 1 shot
Loss:  2.9954288
Accuracy:  40.87496995925903
Model with path ./results/models/omniglot_train_reptile2.h5 loaded.
Testing 20 way 1 shot
Loss:  2.99553
Accuracy:  42.574989795684814
Model with path ./results/models/omniglot_train_reptile3.h5 loaded.
Testing 20 way 1 shot
Loss:  2.9955115
Accuracy:  40.43997526168823


In [ ]:
def build_combined_dataset(train_dataset, val_dataset, test_dataset):
  total_rows = train_dataset.shape[0] + val_dataset.shape[0] + test_dataset.shape[0]
  train_dataset_images = int(train_dataset.shape[1]/2)
  test_dataset_images = int(train_dataset.shape[1]/4)
  val_dataset_images = int(train_dataset.shape[1]/4)
  combined_train_dataset = np.zeros([total_rows, train_dataset_images, img_height, img_width, channels], dtype=np.float32)
  combined_val_dataset = np.zeros([total_rows, val_dataset_images, img_height, img_width, channels], dtype=np.float32)
  combined_test_dataset = np.zeros([total_rows, test_dataset_images, img_height, img_width, channels], dtype=np.float32)

  for i in range(train_dataset.shape[0]):
    for j in range(train_dataset_images):
      combined_train_dataset[i][j] = train_dataset[i][j]
    for k in range(val_dataset_images):
      combined_val_dataset[i][k] = train_dataset[i][train_dataset_images + k]
      combined_test_dataset[i][k] = train_dataset[i][train_dataset_images + val_dataset_images + k]

  for i in range (val_dataset.shape[0]):
    for j in range(train_dataset_images):
      combined_train_dataset[train_dataset.shape[0] + i][j] = val_dataset[i][j]
    for k in range(val_dataset_images):
      combined_val_dataset[train_dataset.shape[0] + i][k] = val_dataset[i][train_dataset_images + k]
      combined_test_dataset[train_dataset.shape[0] + i][k] = train_dataset[i][train_dataset_images + val_dataset_images + k]

  for i in range (test_dataset.shape[0]):
    for j in range(train_dataset_images):
      combined_train_dataset[train_dataset.shape[0] + val_dataset.shape[0] + i][j] = test_dataset[i][j]
    for k in range(val_dataset_images):
      combined_val_dataset[train_dataset.shape[0] + val_dataset.shape[0] + i][k] = test_dataset[i][train_dataset_images + k]
      combined_test_dataset[train_dataset.shape[0] + val_dataset.shape[0] + i][k] = train_dataset[i][train_dataset_images + val_dataset_images + k]

  return combined_train_dataset, combined_val_dataset, combined_test_dataset

In [ ]:
combined_train_dataset, combined_val_dataset, combined_test_dataset = build_combined_dataset(train_dataset, val_dataset, test_dataset)
 

In [ ]:
print(combined_train_dataset.shape)
print(combined_val_dataset.shape)
print(combined_test_dataset.shape)

(6492, 10, 32, 32, 1)
(6492, 5, 32, 32, 1)
(6492, 5, 32, 32, 1)


In [ ]:
combined_train_dataset_reshaped = combined_train_dataset.reshape(64920, 32, 32, 1)
combined_val_dataset_reshaped = combined_val_dataset.reshape(32460, 32, 32, 1)
combined_test_dataset_reshaped = combined_test_dataset.reshape(32460, 32, 32, 1)

In [ ]:
train_labels = np.zeros(combined_train_dataset_reshaped.shape[0])
val_labels = np.zeros(combined_val_dataset_reshaped.shape[0])
test_labels = np.zeros(combined_test_dataset_reshaped.shape[0])
val = 0
for i in range(combined_train_dataset.shape[0]):
  for j in range(combined_train_dataset.shape[1]):
    train_labels[(i * 10) + j] = val
  val = val + 1

val = 0
for i in range(combined_val_dataset.shape[0]):
  for j in range(combined_val_dataset.shape[1]):
    val_labels[(i * 5) + j] = val
    test_labels[(i * 5) + j] = val
  val = val + 1

train_labels[64500:]

array([6450., 6450., 6450., 6450., 6450., 6450., 6450., 6450., 6450.,
       6450., 6451., 6451., 6451., 6451., 6451., 6451., 6451., 6451.,
       6451., 6451., 6452., 6452., 6452., 6452., 6452., 6452., 6452.,
       6452., 6452., 6452., 6453., 6453., 6453., 6453., 6453., 6453.,
       6453., 6453., 6453., 6453., 6454., 6454., 6454., 6454., 6454.,
       6454., 6454., 6454., 6454., 6454., 6455., 6455., 6455., 6455.,
       6455., 6455., 6455., 6455., 6455., 6455., 6456., 6456., 6456.,
       6456., 6456., 6456., 6456., 6456., 6456., 6456., 6457., 6457.,
       6457., 6457., 6457., 6457., 6457., 6457., 6457., 6457., 6458.,
       6458., 6458., 6458., 6458., 6458., 6458., 6458., 6458., 6458.,
       6459., 6459., 6459., 6459., 6459., 6459., 6459., 6459., 6459.,
       6459., 6460., 6460., 6460., 6460., 6460., 6460., 6460., 6460.,
       6460., 6460., 6461., 6461., 6461., 6461., 6461., 6461., 6461.,
       6461., 6461., 6461., 6462., 6462., 6462., 6462., 6462., 6462.,
       6462., 6462.,

In [ ]:
from numpy import argmax
from tensorflow.keras.utils import to_categorical
# one hot encode
train_labels_encoded = to_categorical(train_labels)
val_labels_encoded = to_categorical(val_labels)
test_labels_encoded = to_categorical(test_labels)
print(train_labels_encoded.shape)
print(val_labels_encoded.shape)
print(test_labels_encoded.shape)

(64920, 6492)
(32460, 6492)
(32460, 6492)


In [ ]:
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential

base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,1))
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(6492, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
optimizer = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer= optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(combined_train_dataset_reshaped, train_labels_encoded, epochs = 100, batch_size = 64, validation_data=(combined_val_dataset_reshaped, val_labels_encoded))


Epoch 1/100
1015/1015 [==============================] - 71s 63ms/step - loss: 10.2822 - accuracy: 7.7018e-05 - val_loss: 112.8816 - val_accuracy: 9.2421e-05
Epoch 2/100
1015/1015 [==============================] - 61s 60ms/step - loss: 9.6603 - accuracy: 1.8484e-04 - val_loss: 8.8060 - val_accuracy: 1.5404e-04
Epoch 3/100
1015/1015 [==============================] - 61s 60ms/step - loss: 8.9651 - accuracy: 3.8509e-04 - val_loss: 10.4517 - val_accuracy: 1.8484e-04
Epoch 4/100
1015/1015 [==============================] - 61s 60ms/step - loss: 8.6685 - accuracy: 3.0807e-04 - val_loss: 8.5928 - val_accuracy: 3.0807e-04
Epoch 5/100
1015/1015 [==============================] - 61s 60ms/step - loss: 8.5340 - accuracy: 3.2348e-04 - val_loss: 8.9433 - val_accuracy: 3.3888e-04
Epoch 6/100
1015/1015 [==============================] - 61s 61ms/step - loss: 8.4391 - accuracy: 3.0807e-04 - val_loss: 9.4202 - val_accuracy: 4.0049e-04
Epoch 7/100
1015/1015 [==============================] - 62s 61ms/

In [ ]:
preds = model.evaluate(combined_test_dataset_reshaped, test_labels_encoded)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

1015/1015 [==============================] - 14s 14ms/step - loss: 14.6651 - accuracy: 0.0147
Loss = 14.665081024169922
Test Accuracy = 0.014725816436111927
